In [1]:
import numpy as np
import pandas as pd
import string
from statistics import mean
 
import copy
import os
import re
import torch
import torch.nn as nn
import torch.optim
import nltk
 
from nltk.corpus import stopwords
import spacy
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
import torch.nn.functional as F
 
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
 
!pip install transformers
!pip install pytorch-transformers
 
import transformers
from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from transformers import RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW
 
from tqdm import tqdm, trange
 

     |████████████████████████████████| 1.8MB 18.7MB/s 
     |████████████████████████████████| 890kB 50.0MB/s 
     |████████████████████████████████| 2.9MB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8a2fbfe26caa05fd49c24ea5cba5b69ec0132c7c95e572247713e204aee6b84d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 16.8MB/s 
     |████████████████████████████████| 133kB 42.6MB/s 
     |████████████████████████████████| 1.2MB 47.0MB/s 
     |████████████████████████████████| 7.2MB 49.9MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
ERROR: botocore 1.19.57 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
def clean_data(x):
    puncts = []
    # puncts = puncts.replace("-", "")
    # stop_words = set(stopwords.words('english'))
    sentences = []
    # x = [''.join(c.lower() for c in s.replace("-", " ") if c not in puncts) for s in x]
    x = [''.join(c.lower() for c in s if c not in puncts) for s in x]
    # x = [''.join(c for c in str(s)[1:-4].replace("-", " ") if c not in puncts) for s in x]
    for sent in x:
        # text_no_nums = re.sub(r'\d+', '', sent)
        # text_no_doublespace = re.sub('\s+', ' ', text_no_nums).strip()
        sentences.append(sent)
    return sentences  

def preprocess_targets(y):
    dic = {'NEG': 1, 'POS': 0}
    for t in range(len(y)):
        y[t] = dic[y[t]]
    y = torch.tensor(y, dtype=torch.long)   
    return y  

In [4]:
path = "/content/drive/MyDrive/third_sem/DL_NLP/Project/dataset/CoLA/"

train_path = path + 'train.tsv'
val_path = path + 'dev.tsv'
test_path = path + 'test.tsv'

df = pd.read_csv(train_path, sep='\t', names=['sentence', 'sent_source', 'label', 'label_source'])
print(df[:5])
x_train = df["label_source"].values
y_train = df["sent_source"].values
print("hh",x_train[:10])
print(y_train[:100])

df = pd.read_csv(val_path, sep='\t', names=['sentence', 'sent_source', 'label', 'label_source'])
x_val = df["label_source"].values
y_val = df["sent_source"].values

print(x_val[:5])

# x_val = clean_data(x_val)
# print("hereee",x_val[:2])
# # x_train = [x + " [SEP] [CLS]" for x in x_train]

# df = pd.read_csv(test_path, sep='\t')
# x_test = df["sentence"].values
# print(df)


  sentence  ...                                       label_source
0     gj04  ...  Our friends won't buy this analysis, let alone...
1     gj04  ...  One more pseudo generalization and I'm giving up.
2     gj04  ...   One more pseudo generalization or I'm giving up.
3     gj04  ...     The more we study verbs, the crazier they get.
4     gj04  ...          Day by day the facts are getting murkier.

[5 rows x 4 columns]
hh ["Our friends won't buy this analysis, let alone the next one we propose."
 "One more pseudo generalization and I'm giving up."
 "One more pseudo generalization or I'm giving up."
 'The more we study verbs, the crazier they get.'
 'Day by day the facts are getting murkier.' "I'll fix you a drink."
 'Fred watered the plants flat.'
 'Bill coughed his way out of the restaurant.'
 "We're dancing the night away." 'Herman hammered the metal flat.']
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 

In [5]:
tokenizer_bert = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer_xl = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

In [21]:
def get_ids_and_mask(tokenizer, x, model):
    input_ids = []
    input_mask = []
    token_type_ids = []
    if model == "xl":
        x = [k + " [SEP] [CLS]" for k in x]
    for k in range(len(x)):
        vec = tokenizer.encode_plus(x[k], add_special_tokens=True, max_length=32, pad_to_max_length=True, return_token_type_ids=True)
        input_ids.append(vec['input_ids'])
        input_mask.append(vec['attention_mask'])
        token_type_ids.append(vec['token_type_ids'])

    print(tokenizer.decode(input_ids[0]))
    return input_ids, input_mask, token_type_ids

def dataloader(sentence, mask, token_type_ids, label, d_type, batch_size=32):
    loader = None
    sentence = torch.tensor(sentence)
    mask = torch.tensor(mask)
    token_type_ids = torch.tensor(token_type_ids)
    label = torch.tensor(label)

    print("data shape", sentence.shape)
    print("label shape", label.shape)
    print("mask shape", mask.shape)
    print("tok type id shape", token_type_ids.shape)

    # batch_size = 32

    if d_type == "train":
        train = TensorDataset(sentence, mask, token_type_ids, label)
        sampler = RandomSampler(train)
        loader = DataLoader(train, batch_size=batch_size, sampler=sampler)

    else:
        test = TensorDataset(sentence, mask, token_type_ids, label)
        samp = SequentialSampler(test)
        loader = DataLoader(test, batch_size=batch_size, sampler=samp)

    return loader


In [12]:
###############   TRAINING OF MODEL  ##################

def train_model(model, optim, epochs, loader):
    loss_arr = []
    for ep in range(epochs):
        model.train()
        epoch_loss = 0

        for i, data in enumerate(loader):
            batch_data = data[0].to(device).long()
            batch_mask = data[1].to(device).long()
            # batch_token_type_ids = data[2].to(device).long()
            batch_labels = data[3].to(device).long()

            model.zero_grad()
            pred = model(batch_data,
                                token_type_ids=None,
                                attention_mask=batch_mask,
                                labels=batch_labels)
            
            # As we call the model with labels, it returns the loss in a tuple
            loss = pred[0]
            epoch_loss += loss.item()
            loss.backward()  # Backprpagation

            # Clip Gradient norm to mitigate exploding of gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()

        epoch_loss /= len(loader)
        print("train loss after %d epochs is %f " %(ep+1, epoch_loss))
        loss_arr.append(epoch_loss)

    return loss_arr

######   Validate RoBerta Model   #######

def validate_model(model, loader):
    model.eval()
    pred_list = []
    true_labels = []
    test_acc = 0
    for batch in loader:
        batch = tuple(t for t in batch)
        batch_data, batch_mask, batch_token_type ,batch_labels = batch
        
        batch_data = batch_data.to(device)
        batch_mask = batch_mask.to(device)
        
        with torch.no_grad():
            preds = model(batch_data,
                                    token_type_ids=None,
                                    attention_mask=batch_mask,
                                    labels=batch_labels.to(device))

        logits = preds[1]
        
        logits = logits.detach().cpu()
        targets = batch_labels.to('cpu')
        
        pred_list.append(logits.numpy())
        true_labels.append(targets.numpy())
        
        acc = compute_accuracy(logits, targets)
        test_acc += acc
        # steps += 1

    print("final test set accuracy is ", (test_acc / 1043))
    return pred_list, true_labels
    

In [42]:
####### Evaluating the GLUE score(Mathew's Correlation for CoLA)  ########

def mathews_corr(pred_list, true_labels):
    matthews_set = []
    for i in range(len(true_labels)):
        pred_labels_i = np.argmax(pred_list[i], axis=1).flatten()
    
        # Calculates and stores the coef for this batch.  
        matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
        matthews_set.append(matthews)

    flat_predictions = [item for sublist in pred_list for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

    # Combine the correct labels for each batch into a single list.
    flat_true_labels = [item for sublist in true_labels for item in sublist]
    # Calculate Mathew's correlation
    mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

    print('MCC: %.3f' % mcc)

def compute_accuracy(preds, targets):
    return (torch.argmax(preds, dim=1) == targets).float().sum().item()
    

TILL ABOVE RUN FOR ALL THE 3 APPROACHES

RUN BELOW CELLS FOR APPROACH 1


In [ ]:
####  APPROACH 1 --> prepare dataloader for Roberta and XL Net   #####

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_train, model="bert")
train_dataloader = dataloader(ids, masks, tok_type_ids, y_train, "train")
print()

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_val, model="bert")
val_dataloader = dataloader(ids, masks, tok_type_ids, y_val, "val")

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_xl, x=x_train, model="xl")
train_dataloader_xl = dataloader(ids, masks, tok_type_ids, y_train, "train")
print()

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_xl, x=x_val, model="xl")
val_dataloader_xl = dataloader(ids, masks, tok_type_ids, y_val, "val")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<s>Our friends won't buy this analysis, let alone the next one we propose.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([8551, 32])
label shape torch.Size([8551])
mask shape torch.Size([8551, 32])
tok type id shape torch.Size([8551, 32])



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>The sailors rode the breeze clear of the rocks.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([1043, 32])
label shape torch.Size([1043])
mask shape torch.Size([1043, 32])
tok type id shape torch.Size([1043, 32])
<pad><pad><pad><pad><pad> our friends won't buy this analysis, let alone the next one we propose. [sep] [cls]<sep><cls>
data shape torch.Size([8551, 32])
label shape torch.Size([8551])
mask shape torch.Size([8551, 32])
tok type id shape torch.Size([8551, 32])

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> the sailors rode the breeze clear of the rocks. [sep] [cls]<sep><cls>
data shape torch.Size([1043, 32])
label shape torch.Size([1043])
mask shape torch.Size([1043, 32])
tok type id shape torch.Size([1043, 32])


In [ ]:
######   initialize both models   ######

bert_model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False,
)

bert_model = bert_model.to(device)

xlnet = XLNetForSequenceClassification.from_pretrained(
    "xlnet-base-cased",
    num_labels=2,
)
xlnet = xlnet.to(device)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
optimizer = AdamW(bert_model.parameters(), lr=4e-5, eps=1e-8)
optimizer_xl = AdamW(xlnet.parameters(), lr=4e-5, eps=1e-8)
num_epochs = 4

In [ ]:
#####   APPROACH 1  --> training both models separately   #####

_  = train_model(bert_model, optimizer, epochs=4, loader=train_dataloader)
print()
_ = train_model(xlnet, optimizer_xl, epochs=4, loader=train_dataloader_xl)



train loss after 1 epochs is 0.193445 


In [ ]:
####### Evaluating the GLUE score(Mathew's Correlation for CoLA)  ########

def mathews_corr(pred_list, true_labels):
    matthews_set = []
    for i in range(len(true_labels)):
        pred_labels_i = np.argmax(pred_list[i], axis=1).flatten()
    
        # Calculates and stores the coef for this batch.  
        matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
        matthews_set.append(matthews)

    flat_predictions = [item for sublist in pred_list for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

    # Combine the correct labels for each batch into a single list.
    flat_true_labels = [item for sublist in true_labels for item in sublist]
    # Calculate Mathew's correlation
    mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

    print('MCC: %.3f' % mcc)

In [ ]:
####   Individual results  ####

print("acc using XL NET")
pred_list_1, true_labels_1 = validate_model(xlnet, val_dataloader_xl)
print("acc using Roberta")
pred_list_2, true_labels_2 = validate_model(bert_model, val_dataloader)
mathews_corr(pred_list_1, true_labels_1)
mathews_corr(pred_list_2, true_labels_2)


acc using XL NET
final test set accuracy is  0.7976989453499521
acc using Roberta
final test set accuracy is  0.822627037392138
MCC: 0.498
MCC: 0.566


In [ ]:
######   combine models  --> APPROACH 1    ######

def combine(bert, xlnet, loader_bert, loader_xl):

    bert.eval()
    xlnet.eval()
    test_acc = 0

    for batch, batch_xl in zip(loader_bert, loader_xl):
        batch = tuple(t for t in batch)
        batch_data, batch_mask, batch_token_type ,batch_labels = batch
        
        batch_data = batch_data.to(device)
        batch_mask = batch_mask.to(device)

        batch_xl = tuple(t for t in batch_xl)
        batch_data_xl, batch_mask_xl, batch_token_type_xl ,batch_labels_xl = batch_xl
        
        batch_data_xl = batch_data_xl.to(device)
        batch_mask_xl = batch_mask_xl.to(device)
        
        with torch.no_grad():
            preds1 = bert(batch_data, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels.to(device))
            preds2 = xlnet(batch_data_xl, token_type_ids=None, attention_mask=batch_mask_xl, labels=batch_labels_xl.to(device))
            
        logits1 = preds1[1].detach().to('cpu')
        logits2 = preds2[1].detach().to('cpu')
        targets = batch_labels.to('cpu')
        # print(logits1.shape)

        logits = torch.zeros((logits1.shape))
        for k in range(len(logits1)):
            if torch.argmax(logits1[k]) >= torch.argmax(logits2[k]):
                logits[k] = logits1[k]
            else:
                logits[k] = logits2[k]

        acc = compute_accuracy(logits, targets)
        test_acc += acc

    print("final test set accuracy using approach 1 is ", (test_acc / 1043))

combine(bert_model, xlnet, val_dataloader, val_dataloader_xl)

final test set accuracy using approach 1 is  0.8015340364333653


RUN BELOW CELLS FOR APPROACH 2

In [ ]:
######   APPROACH 2    ########

class ensemble(nn.Module):
    def __init__(self):
        super(ensemble, self).__init__()
        self.bert = transformers.RobertaModel.from_pretrained('roberta-base')
        self.xlnet = transformers.XLNetModel.from_pretrained('xlnet-base-cased')
        self.fc = nn.Linear(768*2, 2)
        self.dropout = nn.Dropout(p=0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, data_bert, mask_bert, data_xl, mask_xl):
        
        out = self.bert(data_bert, attention_mask=mask_bert, token_type_ids=None)
        emb1 = out[0]
        emb1 = torch.mean(emb1, dim=1)
        # print("emb1", emb1.shape)
        
        emb2 = self.xlnet(data_xl, attention_mask=mask_xl, token_type_ids=None)
        emb2 = out[0]
        emb2 = torch.mean(emb2, dim=1)
        # print("emb2", emb2.shape)

        emb = torch.cat((emb1, emb2), dim=1)

        return self.softmax(self.fc(emb))


In [ ]:
def train_ensemble(model, loader, optim):
    loss_arr = []
    for ep in range(4):
        epoch_loss = 0
        for i, data in enumerate(loader):
            model.train()
            batch_data = data[0].to(device).long()
            batch_mask = data[1].to(device).long()
            batch_labels = data[6].to(device).long()

            batch_data_xl = data[3].to(device).long()
            batch_mask_xl = data[4].to(device).long()

            pred = model(batch_data, batch_mask, batch_data_xl, batch_mask_xl)
            optim.zero_grad()
            
            loss = F.cross_entropy(pred, batch_labels)
            epoch_loss += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()

        epoch_loss /= len(loader)
        print("train loss after %d epochs is %f " %(ep+1, epoch_loss))
        loss_arr.append(epoch_loss)
    return loss_arr


def validate_ensemble(model, loader):
    model.eval()
    pred_list = []
    true_labels = []
    test_acc = 0
    for batch in loader:
        batch = tuple(t for t in batch)
        batch_data, batch_mask, z1 , batch_data_xl, batch_mask_xl, z2, batch_labels = batch
        
        batch_data = batch_data.to(device)
        batch_mask = batch_mask.to(device)
        batch_data_xl = batch_data_xl.to(device)
        batch_mask_xl = batch_mask_xl.to(device)
        
        with torch.no_grad():
            logits = model(batch_data, batch_mask, batch_data_xl, batch_mask_xl)

        # logits = preds[1]
        
        logits = logits.detach().cpu()
        targets = batch_labels.to('cpu')
        
        pred_list.append(logits.numpy())
        true_labels.append(targets.numpy())
        
        acc = compute_accuracy(logits, targets)
        test_acc += acc
        # steps += 1

    print("final test set accuracy is ", (test_acc / 1043))
    return pred_list, true_labels


In [ ]:
def dataloader_comb(sent1, sent2, mask1, mask2, id1, id2, label, d_type, batch_size=32):
    loader = None
    sent1 = torch.tensor(sent1)
    sent2 = torch.tensor(sent2)
    mask1 = torch.tensor(mask1)
    mask2 = torch.tensor(mask2)
    id1 = torch.tensor(id1)
    id2 = torch.tensor(id2)
    label = torch.tensor(label)

    # print("data shape", sentence.shape)
    # print("label shape", label.shape)
    # print("mask shape", mask.shape)
    # print("tok type id shape", token_type_ids.shape)

    batch_size = 32

    if d_type == "train":
        train = TensorDataset(sent1, mask1, id1, sent2, mask2, id2, label)
        sampler = RandomSampler(train)
        loader = DataLoader(train, batch_size=batch_size, sampler=sampler)

    else:
        test = TensorDataset(sent1, mask1, id1, sent2, mask2, id2, label)
        samp = SequentialSampler(test)
        loader = DataLoader(test, batch_size=batch_size, sampler=samp)

    return loader

In [ ]:

####   prepare dataloader for combined model   #####

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_train, model="bert")
ids1, masks1, tok_type_ids1 = get_ids_and_mask(tokenizer=tokenizer_xl, x=x_train, model="xl")
train_dataloader = dataloader_comb(ids, ids1, masks, masks1, tok_type_ids, tok_type_ids1, y_train, "train")
print()

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_val, model="bert")
ids1, masks1, tok_type_ids1 = get_ids_and_mask(tokenizer=tokenizer_xl, x=x_val, model="xl")
val_dataloader = dataloader_comb(ids, ids1, masks, masks1, tok_type_ids, tok_type_ids1, y_val, "val")


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<s>Our friends won't buy this analysis, let alone the next one we propose.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<pad><pad><pad><pad><pad> our friends won't buy this analysis, let alone the next one we propose. [sep] [cls]<sep><cls>

<s>The sailors rode the breeze clear of the rocks.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> the sailors rode the breeze clear of the rocks. [sep] [cls]<sep><cls>


In [ ]:
ensemble_model = ensemble().to(device)
optimizer = AdamW(ensemble_model.parameters(), lr=4e-5, eps=1e-10)


In [ ]:
#####   call train ensemble   ######

_ = train_ensemble(ensemble_model, train_dataloader, optimizer)
l, m = validate_ensemble(ensemble_model, val_dataloader)

final test set accuracy is  0.7986577181208053


RUN BELOW CELLS FOR APPROACH 3

In [15]:
####  APPROACH 3 starts here   ####

### prepare data to pass to model  ##

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_train, model="bert")
train_dataloader = dataloader(ids, masks, tok_type_ids, y_train, "train", batch_size=32)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<s>Our friends won't buy this analysis, let alone the next one we propose.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([8551, 32])
label shape torch.Size([8551])
mask shape torch.Size([8551, 32])
tok type id shape torch.Size([8551, 32])


In [16]:
bert_model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=2,
)
bert_model = bert_model.to(device)
optimizer = AdamW(bert_model.parameters(), lr=4e-5, eps=1e-10)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [17]:
###  train model which will score examples  ##

_ = train_model(bert_model, optimizer, epochs=4, loader=train_dataloader)

train loss after 1 epochs is 0.512399 
train loss after 2 epochs is 0.337831 
train loss after 3 epochs is 0.245066 
train loss after 4 epochs is 0.179049 


In [29]:
###   prepare train data to get loss om each  ###

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_train, model="bert")
train_dataloader = dataloader(ids, masks, tok_type_ids, y_train, "val", batch_size=1)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<s>Our friends won't buy this analysis, let alone the next one we propose.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([8551, 32])
label shape torch.Size([8551])
mask shape torch.Size([8551, 32])
tok type id shape torch.Size([8551, 32])


In [30]:
#####   Approach  3   get loss on all training data   #####

bert_model.eval()
loss_list = []
for i, data in enumerate(train_dataloader):
    batch_data = data[0].to(device).long()
    batch_mask = data[1].to(device).long()
    batch_labels = data[3].to(device).long()

    with torch.no_grad():
        pred = bert_model(batch_data,
                                token_type_ids=None,
                                attention_mask=batch_mask,
                                labels=batch_labels)
        
        
    loss = pred[0]
    loss_list.append(loss.item())
    

In [31]:
########     SORT EXAMPLES BASED ON TOUGHNESS     ######

print(max(loss_list))
print(min(loss_list))

train_loss = []

for k in range(len(x_train)):
    train_loss.append((x_train[k], y_train[k], loss_list[k]))
    # label_loss.append((y_train[k], loss_list[k]))

print(train_loss[-1])
train_loss = sorted(train_loss, key = lambda x: x[2])

print(train_loss[-1])

x_train_sorted = []
y_train_sorted = []

for k in train_loss:
    x_train_sorted.append(k[0])
    y_train_sorted.append(k[1])

print(x_train_sorted[-1])
print(len(x_train_sorted), len(y_train_sorted))

5.133019924163818
0.001861388562247157
('What all did you get for Christmas?', 1, 0.005805535241961479)
('Dorothy is needing new shoes.', 0, 5.133019924163818)
Dorothy is needing new shoes.
8551 8551


In [45]:
####   NOW Preapare data in curriculum order from sorted examples  ####

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_train_sorted, model="bert")
train_dataloader = dataloader(ids, masks, tok_type_ids, y_train_sorted, "train", batch_size=32)

ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_val, model="bert")
val_dataloader = dataloader(ids, masks, tok_type_ids, y_val, "val", batch_size=32)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<s>I put the book from Edna.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([8551, 32])
label shape torch.Size([8551])
mask shape torch.Size([8551, 32])
tok type id shape torch.Size([8551, 32])
<s>The sailors rode the breeze clear of the rocks.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([1043, 32])
label shape torch.Size([1043])
mask shape torch.Size([1043, 32])
tok type id shape torch.Size([1043, 32])


In [46]:
###  model for training in curriculum oreder  ###

new_model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False,
)

new_model = new_model.to(device)
optim = AdamW(new_model.parameters(), lr=4e-5, eps=1e-8)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [48]:
###  train model  ####

arr = train_model(new_model, optim, 2, train_dataloader)
ids, masks, tok_type_ids = get_ids_and_mask(tokenizer=tokenizer_bert, x=x_train, model="bert")
train_dataloader = dataloader(ids, masks, tok_type_ids, y_train, "train", batch_size=32)
arr = train_model(new_model, optim, 2, train_dataloader)

train loss after 1 epochs is 0.494032 
train loss after 2 epochs is 0.323597 


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<s>Our friends won't buy this analysis, let alone the next one we propose.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
data shape torch.Size([8551, 32])
label shape torch.Size([8551])
mask shape torch.Size([8551, 32])
tok type id shape torch.Size([8551, 32])
train loss after 1 epochs is 0.234240 
train loss after 2 epochs is 0.180365 


In [49]:
_,_ = validate_model(new_model, val_dataloader)

final test set accuracy is  0.8053691275167785
